# Quadrotor Simulator

In order to easily simulate different algorithms, a simulation test bed must be created. This Python class deals with the plumbing between the controller, estimation, and the model of a quadrotor system. This notebook also provides abstract base classes for `Controller` and `Estimator`.

In [1]:
%%capture
# Load the quadrotor simulator from a previous notebook
%run quadrotor_model.ipynb

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class Controller(object):
    """Controller
    """
    def __init__(self):
        self.name = "Manual"
    
    def __str__(self):
        return self.name
    
    def update(self, state, Ts):
        return np.array([[10, 0, 0, 0]]).T

In [4]:
class Estimator(object):
    """Estimator
    """
    def __init__(self):
        self.use_truth = True
        self.name = "Truth"
        
    def __str__(self):
        return self.name
    
    def update(self, Ts):
        pass

In [5]:
class Simulator(object):
    """Simulator
        
    This class deals with the high-level simulation plumbing for a quadrotor system
    """
    def __init__(self, quad=None, ctrl=None, estm=None):
        self.quad = quad if quad else Quadrotor()
        self.ctrl = ctrl if ctrl else Controller()
        self.estm = estm if estm else Estimator()
        
        # Keep a history for plotting
        self.hist = { 'u': None, 'state': None }
        
    def __str__(self):
        s  = "Simulation"
        return s
    
    def _step(self):
        pass
    
    def run(self, Tf, Ts=0.01):
        
        # How many iterations are needed
        N = int(Tf/Ts)
        
        # quadrotor state
        state = np.zeros((12,1))
        state[0:3] = self.quad.r
        state[3:6] = self.quad.v
        state[6:9] = self.quad.Phi
        state[9:12] = self.quad.omega
        
        # initialize the plot history
        
        for i in range(N):
            # determine control
            u = self.ctrl.update(state, Ts)
                      
            # actuate physical model
            self.quad.update(u, Ts)
            
            # run estimator
            if self.estm.use_truth:
                state[0:3] = self.quad.r
                state[3:6] = self.quad.v
                state[6:9] = self.quad.Phi
                state[9:12] = self.quad.omega
            else:
                state = self.estm.update(Ts)
                
            # Update history
            
    
    def plot(self):
        pass

In [9]:
sim = Simulator()
sim.run(5, Ts=0.01)
sim.plot()

print(sim.quad)

Quadrotor state after 500 iters:
	r: [[ 0.       0.      -2.37025]].T
	Phi: [[ 0.  0.  0.]].T
	v: [[ 0.    0.   -0.95]].T
	omega: [[ 0.  0.  0.]].T

